In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar  4 06:43:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.7 MB/s eta 0:00:00


In [4]:
import os 
import numpy as np
import pandas as pd
import json
import warnings
import logging
import gc
import random
import math
import re
import ast
from tqdm import tqdm
from typing import Optional
from datetime import datetime
import pickle
import sklearn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [5]:
import pickle
import torch
import transformers
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer, AutoModel, AutoConfig




import torch.nn.functional as F
import numpy as np

import math
import pandas as pd
import pip
import re

import os.path as osp



import random

random.seed(123)

torch.manual_seed(123)
np.random.seed(123)
torch.cuda.manual_seed_all(123)
# from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix


In [6]:
with open('/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/data_dict.p', 'rb') as f:
  data_dict = pickle.load(f)

In [7]:
train_data = data_dict['train_data']
train_label = data_dict['train_label']
valid_data = data_dict['valid_data']
valid_label = data_dict['valid_label']
test_data = data_dict['test_data']
test_label = data_dict['test_label']

In [8]:
train_data[0]

'[PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SEP] [DOC] : Per your symptom, you may have [MASK]. '

In [9]:

tokenizer = AutoTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')
language_model = AutoModel.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')

In [10]:
language_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
tokenizer

BertTokenizerFast(name_or_path='cambridgeltl/SapBERT-from-PubMedBERT-fulltext', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [12]:
class CustomModel(torch.nn.Module):
  def __init__(self, hidden_dim, output_dim):
    super(CustomModel, self).__init__()

    self.custom_model = language_model
    self.dense = torch.nn.Linear(hidden_dim, output_dim)

  def forward(self, input_ids, attention_mask):
    model_out = self.custom_model(input_ids, attention_mask)['last_hidden_state'][:, 0, :]
    dense_out = self.dense(model_out)
    return dense_out  


In [13]:
model = CustomModel(hidden_dim = 768, output_dim = 90)
# model = CustomModel(hidden_dim = 1024, output_dim = 90)

In [14]:

print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]']
    }

tokenizer.add_special_tokens(p)
print(tokenizer.convert_tokens_to_ids('[DOC]'))
print(tokenizer.convert_tokens_to_ids('[PAT]'))
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(language_model.resize_token_embeddings(len(tokenizer)))

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]
30522
30523
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]']
[1, 3, 0, 2, 4, 30522, 30523]
Embedding(30524, 768)


In [15]:
enc_train = tokenizer(train_data, padding='max_length', max_length=512, truncation=True, return_tensors = 'pt')
enc_valid = tokenizer(valid_data, padding='max_length', max_length=512, truncation=True, return_tensors = 'pt')
enc_test = tokenizer(test_data, padding='max_length', max_length=512, truncation=True, return_tensors = 'pt')


In [16]:
train_label = torch.tensor(train_label)
print(train_label.shape)

valid_label = torch.tensor(valid_label)
print(valid_label.shape)

test_label = torch.tensor(test_label)
print(test_label.shape)

torch.Size([957])
torch.Size([137])
torch.Size([273])


In [17]:
class MedicalDataset(Dataset):
  def __init__(self, input_ids, attention_mask, labels):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.labels = labels

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]    

In [18]:
train_loader = DataLoader(MedicalDataset(enc_train['input_ids'], enc_train['attention_mask'], train_label), batch_size = 16, shuffle = True )
valid_loader = DataLoader(MedicalDataset(enc_valid['input_ids'], enc_valid['attention_mask'], valid_label), batch_size = 16, shuffle = False)
test_loader = DataLoader(MedicalDataset(enc_test['input_ids'], enc_test['attention_mask'], test_label), batch_size = 16, shuffle = False)

In [19]:
model

CustomModel(
  (custom_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
model = model.to(device)

In [22]:
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_parameters/1e6

109.552986

In [23]:
cnt = 0
for name, parma in model.named_parameters():
  print(cnt, " : ", name)
  cnt+=1

0  :  custom_model.embeddings.word_embeddings.weight
1  :  custom_model.embeddings.position_embeddings.weight
2  :  custom_model.embeddings.token_type_embeddings.weight
3  :  custom_model.embeddings.LayerNorm.weight
4  :  custom_model.embeddings.LayerNorm.bias
5  :  custom_model.encoder.layer.0.attention.self.query.weight
6  :  custom_model.encoder.layer.0.attention.self.query.bias
7  :  custom_model.encoder.layer.0.attention.self.key.weight
8  :  custom_model.encoder.layer.0.attention.self.key.bias
9  :  custom_model.encoder.layer.0.attention.self.value.weight
10  :  custom_model.encoder.layer.0.attention.self.value.bias
11  :  custom_model.encoder.layer.0.attention.output.dense.weight
12  :  custom_model.encoder.layer.0.attention.output.dense.bias
13  :  custom_model.encoder.layer.0.attention.output.LayerNorm.weight
14  :  custom_model.encoder.layer.0.attention.output.LayerNorm.bias
15  :  custom_model.encoder.layer.0.intermediate.dense.weight
16  :  custom_model.encoder.layer.0.inte

In [24]:
cnt = 0
for name, param in model.named_parameters():
  if(cnt>=199):
  # if(cnt>=391):  
    param.requires_grad = True
    print(cnt, " : ", name)
  else:
    param.requires_grad = False 

  cnt+=1   


199  :  dense.weight
200  :  dense.bias


In [25]:
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_parameters/1e6

0.06921

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()


In [27]:
def train_epoch(model, data_loader):
  model.train()
  epoch_train_loss = 0.0

  for step, batch in enumerate(tqdm(data_loader)):
    optimizer.zero_grad()

    batch = tuple(t.to(device) for t in batch)

    input_ids, attention_mask, labels = batch

    output = model(input_ids = input_ids,
                   attention_mask = attention_mask)
    
    loss = criterion(output, labels.long())

    epoch_train_loss += loss.item()

    loss.backward()
    optimizer.step()

  print("Epoch train loss : ", epoch_train_loss)    


  

In [28]:
def valid_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  valid_loss = 0.0

  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(device) for t in batch)

      input_ids, attention_mask, labels = batch

      output = model(input_ids = input_ids,
                   attention_mask = attention_mask)
    
      loss = criterion(output, labels.long())

      valid_loss += loss.item()

      pred = output.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())

  return valid_loss, predictions, gold    



In [29]:
def test_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(device) for t in batch)

      input_ids, attention_mask, labels = batch

      output = model(input_ids = input_ids,
                   attention_mask = attention_mask)
      
      pred = output.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())


  return predictions, gold    





In [30]:
def train_and_validation(model, train_loader, valid_loader):
  best_acc = 0.0
  

  for epoch in range(50):

    print("\n=============Epoch : ", epoch)
    train_epoch(model, train_loader)
    valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

    print("Length of predictions : ", len(valid_pred))
    print("Length of gold : ", len(valid_gold))
    print("Valid loss : ", valid_loss)
    
    print("\nAccuracy\n", accuracy_score(valid_gold, valid_pred))
    print("\nF1 score\n", f1_score(valid_gold, valid_pred, average = 'macro'))
    print("\nJaccard similarity\n", jaccard_score(valid_gold, valid_pred, average = 'macro')) 

    curr_acc = accuracy_score(valid_gold, valid_pred)

    print('Curr acc : ', curr_acc)
    print('Best acc : ', best_acc)

    if(curr_acc > best_acc):
      best_acc = curr_acc

      
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/sapbert/best_model_epoch_'+str(epoch)+'_best_acc_'+str(int(best_acc*100))+'_.pt')

      print("model saved\n")

In [31]:
train_and_validation(model, train_loader, valid_loader)


=============Epoch :  0


100%|██████████| 60/60 [00:10<00:00,  5.78it/s]


Epoch train loss :  244.81377530097961


100%|██████████| 9/9 [00:00<00:00,  9.19it/s]


Length of predictions :  137
Length of gold :  137
Valid loss :  29.771571397781372

Accuracy
 0.32116788321167883

F1 score
 0.19669209789178832

Jaccard similarity
 0.15411184210526313
Curr acc :  0.32116788321167883
Best acc :  0.0
model saved


=============Epoch :  1


...

Length of predictions :  137
Length of gold :  137
Valid loss :  11.049973368644714

Accuracy
 0.6496350364963503

F1 score
 0.5481097961867193

Jaccard similarity
 0.4832264957264958
Curr acc :  0.6496350364963503
Best acc :  0.656934306569343

=============Epoch :  49


100%|██████████| 60/60 [00:07<00:00,  8.28it/s]


Epoch train loss :  6.4491177052259445


100%|██████████| 9/9 [00:00<00:00,  9.19it/s]

Length of predictions :  137
Length of gold :  137
Valid loss :  11.138934761285782

Accuracy
 0.6204379562043796

F1 score
 0.5149267399267399

Jaccard similarity
 0.45028490028490026
Curr acc :  0.6204379562043796
Best acc :  0.656934306569343


In [32]:
path = '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/sapbert/best_model_epoch_35_best_acc_65_.pt'

In [33]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [34]:
test_pred, test_gold = test_epoch(model, test_loader)

print("\nAccuracy\n", accuracy_score(test_gold, test_pred))
print("\nF1 score\n", f1_score(test_gold, test_pred, average = 'macro'))
print("\nJaccard similarity\n", jaccard_score(test_gold, test_pred, average = 'macro')) 

100%|██████████| 18/18 [00:02<00:00,  8.94it/s]


Accuracy
 0.6153846153846154

F1 score
 0.5801639332889333

Jaccard similarity
 0.48341901154401157
